In [ ]:
import pandas as pd

chunksize = 10

bams = pd.read_csv('s3://olgabot-maca/facs_bams.csv')
# bams = bams.rename(columns={'sample_id': 'sample_name'})
print(bams.shape)
bams.head()

In [ ]:
import numpy as np

In [ ]:
bams.head()

In [ ]:
chunksize = 10

groupby = np.arange(len(bams))//10

def pipe_join(names):
    return "|".join(names)

new_dfs = []

sample_names = bams.groupby(groupby)['sample_id'].apply(pipe_join)
new_bams = bams.groupby(groupby)['bam'].apply(pipe_join)

reflow_bams = pd.concat({"sample_names":sample_names, "bams":new_bams}, axis=1)
reflow_bams['output'] = f's3://olgabot-maca/facs/merge_tabula_muris/merged_chunksize={chunksize}/merged_' + reflow_bams.index.astype(str) + '.bam'
reflow_bams.index.name = 'id'
reflow_bams.head()

In [ ]:
samples = reflow_bams

In [ ]:
import json

folder = '../merge_chunksize={chunksize}'

! mkdir -p $folder
assert samples.index.is_unique
samples.to_csv(f'{folder}/samples.csv')

config = 	{
		"program": "../reflow/merge_bams.rf",
		"runs_file": "samples.csv"
	}

with open(f'{folder}/config.json', 'w') as f:
    json.dump(config, f)
    
! head -n 2 $folder/samples.csv $folder/config.json